# Phase 1 — Data Preparation & Preprocessing (Advanced_SIEM_Dataset)

In [33]:
from pathlib import Path
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)

HERE = Path.cwd()
PROJECT_ROOT = HERE.parent if HERE.name == "Notebooks" else HERE

RAW_DIR  = PROJECT_ROOT / "data" / "raw"
PREP_DIR = PROJECT_ROOT / "data" / "processed"
OUT_DIR  = PROJECT_ROOT / "results"
MODEL_DIR = PROJECT_ROOT / "models" / "preprocess"

for p in [RAW_DIR, PREP_DIR, OUT_DIR, MODEL_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("RAW_DIR:", RAW_DIR)
print("PREP_DIR:", PREP_DIR)
print("OUT_DIR:", OUT_DIR)
print("MODEL_DIR:", MODEL_DIR)


PROJECT_ROOT: d:\INS\Assignment_2
RAW_DIR: d:\INS\Assignment_2\data\raw
PREP_DIR: d:\INS\Assignment_2\data\processed
OUT_DIR: d:\INS\Assignment_2\results
MODEL_DIR: d:\INS\Assignment_2\models\preprocess


In [34]:
from huggingface_hub import hf_hub_download

REPO_ID = "darkknight25/Advanced_SIEM_Dataset"
FILENAME = "advanced_siem_dataset.jsonl"

raw_jsonl_path = Path(hf_hub_download(
    repo_id=REPO_ID,
    filename=FILENAME,
    repo_type="dataset",
    local_dir=str(RAW_DIR),
    local_dir_use_symlinks=False,   # makes a real copy inside your RAW_DIR
))

raw_jsonl_path


C:\Users\Carol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\huggingface_hub\utils\_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(


WindowsPath('d:/INS/Assignment_2/data/raw/advanced_siem_dataset.jsonl')

In [35]:
df_jsonl = pd.read_json(raw_jsonl_path, lines=True)
print("JSONL shape:", df_jsonl.shape)
df_jsonl.head(3)


JSONL shape: (100000, 35)


,event_id,timestamp,event_type,source,severity,raw_log,advanced_metadata,user,action,object,process_id,parent_process,additional_info,description,behavioral_analytics,device_type,device_id,firmware_version,src_ip,dst_ip,alert_type,signature_id,category,cloud_service,resource_id,model_id,input_hash,output_hash,src_port,dst_port,protocol,bytes,duration,method,mac_address
0,8e785e09-5213-46b1-a6eb-b7e40998905b,2025-05-28 23:46:49,endpoint,Microsoft Sentinel v1.0.0,critical,CEF:0|Microsoft Sentinel v1.0.0|SIEM|1.0|100|e...,"{'geo_location': 'Isle of Man', 'device_hash':...",deannataylor,file_access,/I/fear.ppt,8141.0,explorer.exe,No additional info,Endpoint file_access /I/fear.ppt by deannatayl...,"{'baseline_deviation': 1.8399999999999999, 'en...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bf4fa0a9-0665-40cd-ad81-6bdc84f189d4,2025-01-22 04:17:38,iot,AlienVault v5.7.0,low,CEF:0|AlienVault v5.7.0|SIEM|1.0|100|iot|low| ...,"{'geo_location': 'Faroe Islands', 'device_hash...",NaN,side_channel,NaN,NaN,NaN,No additional info,IoT device HVAC side_channel No additional info,NaN,HVAC,iot-dc0f5947,9.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e400e1b2-d174-43d0-8a17-f1f966a2b857,2025-03-21 10:03:20,ids_alert,Carbon Black v7.8.0,critical,CEF:0|Carbon Black v7.8.0|SIEM|1.0|100|ids_ale...,"{'geo_location': 'Mexico', 'device_hash': '63c...",NaN,NaN,NaN,NaN,NaN,No additional info,Carbon Black Alert: Credential Stuffing detect...,NaN,NaN,NaN,NaN,54.159.34.148,N/A,Credential Stuffing,SIG-9267,Exploit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Flatten nested advanced_metadata if present (common in this dataset)
if "advanced_metadata" in df_jsonl.columns:
    meta = pd.json_normalize(df_jsonl["advanced_metadata"]).add_prefix("meta_")
    df_jsonl = pd.concat([df_jsonl.drop(columns=["advanced_metadata"]), meta], axis=1)

# Simple mapping into 3 sources (Firewall, IDS, System)
# (dataset has event_type categories like firewall / ids_alert / auth / endpoint / etc.)
ids_df = df_jsonl[df_jsonl["event_type"] == "ids_alert"].copy()
fw_df  = df_jsonl[df_jsonl["event_type"].isin(["firewall", "network"])].copy()
sys_df = df_jsonl[df_jsonl["event_type"].isin(["auth", "endpoint", "cloud", "iot", "ai"])].copy()

ids_path = RAW_DIR / "ids_logs.csv"
fw_path  = RAW_DIR / "firewall_logs.csv"
sys_path = RAW_DIR / "system_logs.csv"

ids_df.to_csv(ids_path, index=False)
fw_df.to_csv(fw_path, index=False)
sys_df.to_csv(sys_path, index=False)

print("Saved:", ids_path, fw_path, sys_path)
print("Counts:", len(ids_df), len(fw_df), len(sys_df))


Saved: d:\INS\Assignment_2\data\raw\ids_logs.csv d:\INS\Assignment_2\data\raw\firewall_logs.csv d:\INS\Assignment_2\data\raw\system_logs.csv
Counts: 12500 24783 62717


In [37]:
# Data Loading with read_csv() (Required: Step 1.1)

ids_logs = pd.read_csv(RAW_DIR / "ids_logs.csv")
fw_logs  = pd.read_csv(RAW_DIR / "firewall_logs.csv")
sys_logs = pd.read_csv(RAW_DIR / "system_logs.csv")

ids_logs["log_source"] = "IDS"
fw_logs["log_source"]  = "FIREWALL"
sys_logs["log_source"] = "SYSTEM"

print(ids_logs.shape, fw_logs.shape, sys_logs.shape)


(12500, 41) (24783, 41) (62717, 41)


In [38]:
# Create unified dataframe by “merging” on timestamp/IP/host (Required: Step 1.1)

df = pd.concat([ids_logs, fw_logs, sys_logs], ignore_index=True)

# Normalize timestamps (single timezone) :contentReference[oaicite:7]{index=7}
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=True)

# Make sure required merge fields exist
for c in ["host", "src_ip", "dst_ip"]:
    if c not in df.columns:
        df[c] = "unknown"

df["host"] = df["host"].fillna("unknown")
df["src_ip"] = df["src_ip"].fillna("0.0.0.0")
df["dst_ip"] = df["dst_ip"].fillna("0.0.0.0")

# timestamp rounding (nearest-match / rounding)
df["ts_round"] = df["timestamp"].dt.round("1s")

# merge key: ts + src/dst + host
df["merge_key"] = (
    df["ts_round"].astype(str) + "|" +
    df["host"].astype(str) + "|" +
    df["src_ip"].astype(str) + "|" +
    df["dst_ip"].astype(str)
)

# unified dataframe (all events)
df = df.sort_values("timestamp").reset_index(drop=True)
df.head(3)
# print(df.shape)


,event_id,timestamp,event_type,source,severity,raw_log,user,action,object,process_id,parent_process,additional_info,description,behavioral_analytics,device_type,device_id,firmware_version,src_ip,dst_ip,alert_type,signature_id,category,cloud_service,resource_id,model_id,input_hash,output_hash,src_port,dst_port,protocol,bytes,duration,method,mac_address,meta_geo_location,meta_device_hash,meta_user_agent,meta_session_id,meta_risk_score,meta_confidence,log_source,host,ts_round,merge_key
0,8f7b1e90-e534-4a0b-b273-c58ce683dafe,2020-07-12 21:38:20+00:00,firewall,CrowdStrike v6.45.0,info,CEF:0|CrowdStrike v6.45.0|SIEM|1.0|100|firewal...,NaN,deny,NaN,NaN,NaN,Associated Threat Actor: APT29,Firewall deny ICMP traffic from 197.198.51.199...,NaN,NaN,NaN,NaN,197.198.51.199,182.162.163.189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,845.0,856.0,ICMP,626084.0,2859.0,NaN,NaN,Bhutan,c86a1842266ff27035c9b0a28110a26d56801ce7,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_5_7 ...,4ff151bb-32a0-4cfc-9b8e-e38d70faba9b,62.23,0.60,FIREWALL,unknown,2020-07-12 21:38:20+00:00,2020-07-12 21:38:20+00:00|unknown|197.198.51.1...
1,168ca1a1-e358-41d5-95e2-a0c747c6bc22,2020-07-14 14:15:20+00:00,network,Carbon Black v7.8.0,info,CEF:0|Carbon Black v7.8.0|SIEM|1.0|100|network...,NaN,latency_spike,NaN,NaN,NaN,No additional info,Network latency_spike from 33.122.86.82 to 38....,NaN,NaN,NaN,NaN,33.122.86.82,38.74.223.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UDP,352862.0,NaN,NaN,NaN,Vietnam,614ffe675a960a828e11c31d43750314d595722c,Mozilla/5.0 (Windows NT 5.0) AppleWebKit/535.1...,23a671d9-444b-48e0-905d-701cac2a3dda,37.69,0.97,FIREWALL,unknown,2020-07-14 14:15:20+00:00,2020-07-14 14:15:20+00:00|unknown|33.122.86.82...
2,77713c17-6a3d-40fc-a824-de4addaaf692,2020-07-15 12:06:45+00:00,cloud,Carbon Black v7.8.0,low,CEF:0|Carbon Black v7.8.0|SIEM|1.0|100|cloud|l...,smithheather,config_change,NaN,NaN,NaN,No additional info,Cloud config_change in GCP by smithheather No ...,NaN,NaN,NaN,NaN,0.0.0.0,0.0.0.0,NaN,NaN,NaN,GCP,res-a77d35de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pitcairn Islands,fb5d0c7b7fc02c6fbc49f6b90d2870b40853cc855d7463...,Mozilla/5.0 (compatible; MSIE 6.0; Windows 98;...,529948a7-5777-4e4b-a6e5-6f83501519f8,34.61,0.80,SYSTEM,unknown,2020-07-15 12:06:45+00:00,2020-07-15 12:06:45+00:00|unknown|0.0.0.0|0.0.0.0


In [39]:
# Inspect data (Required: Step 1.1)

df.info()
display(df.head())
display(df.describe(include="all"))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 44 columns):
 #   Column                Non-Null Count   Dtype              
---  ------                --------------   -----              
 0   event_id              100000 non-null  object             
 1   timestamp             100000 non-null  datetime64[ns, UTC]
 2   event_type            100000 non-null  object             
 3   source                100000 non-null  object             
 4   severity              100000 non-null  object             
 5   raw_log               100000 non-null  object             
 6   user                  50283 non-null   object             
 7   action                87500 non-null   object             
 8   object                12589 non-null   object             
 9   process_id            12589 non-null   float64            
 10  parent_process        12589 non-null   object             
 11  additional_info       100000 non-null  object        

,event_id,timestamp,event_type,source,severity,raw_log,user,action,object,process_id,parent_process,additional_info,description,behavioral_analytics,device_type,device_id,firmware_version,src_ip,dst_ip,alert_type,signature_id,category,cloud_service,resource_id,model_id,input_hash,output_hash,src_port,dst_port,protocol,bytes,duration,method,mac_address,meta_geo_location,meta_device_hash,meta_user_agent,meta_session_id,meta_risk_score,meta_confidence,log_source,host,ts_round,merge_key
0,8f7b1e90-e534-4a0b-b273-c58ce683dafe,2020-07-12 21:38:20+00:00,firewall,CrowdStrike v6.45.0,info,CEF:0|CrowdStrike v6.45.0|SIEM|1.0|100|firewal...,NaN,deny,NaN,NaN,NaN,Associated Threat Actor: APT29,Firewall deny ICMP traffic from 197.198.51.199...,NaN,NaN,NaN,NaN,197.198.51.199,182.162.163.189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,845.0,856.0,ICMP,626084.0,2859.0,NaN,NaN,Bhutan,c86a1842266ff27035c9b0a28110a26d56801ce7,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_5_7 ...,4ff151bb-32a0-4cfc-9b8e-e38d70faba9b,62.23,0.60,FIREWALL,unknown,2020-07-12 21:38:20+00:00,2020-07-12 21:38:20+00:00|unknown|197.198.51.1...
1,168ca1a1-e358-41d5-95e2-a0c747c6bc22,2020-07-14 14:15:20+00:00,network,Carbon Black v7.8.0,info,CEF:0|Carbon Black v7.8.0|SIEM|1.0|100|network...,NaN,latency_spike,NaN,NaN,NaN,No additional info,Network latency_spike from 33.122.86.82 to 38....,NaN,NaN,NaN,NaN,33.122.86.82,38.74.223.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UDP,352862.0,NaN,NaN,NaN,Vietnam,614ffe675a960a828e11c31d43750314d595722c,Mozilla/5.0 (Windows NT 5.0) AppleWebKit/535.1...,23a671d9-444b-48e0-905d-701cac2a3dda,37.69,0.97,FIREWALL,unknown,2020-07-14 14:15:20+00:00,2020-07-14 14:15:20+00:00|unknown|33.122.86.82...
2,77713c17-6a3d-40fc-a824-de4addaaf692,2020-07-15 12:06:45+00:00,cloud,Carbon Black v7.8.0,low,CEF:0|Carbon Black v7.8.0|SIEM|1.0|100|cloud|l...,smithheather,config_change,NaN,NaN,NaN,No additional info,Cloud config_change in GCP by smithheather No ...,NaN,NaN,NaN,NaN,0.0.0.0,0.0.0.0,NaN,NaN,NaN,GCP,res-a77d35de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pitcairn Islands,fb5d0c7b7fc02c6fbc49f6b90d2870b40853cc855d7463...,Mozilla/5.0 (compatible; MSIE 6.0; Windows 98;...,529948a7-5777-4e4b-a6e5-6f83501519f8,34.61,0.80,SYSTEM,unknown,2020-07-15 12:06:45+00:00,2020-07-15 12:06:45+00:00|unknown|0.0.0.0|0.0.0.0
3,c1ceab28-b9a7-4ff2-9f7e-d6bfa9a506dc,2020-07-16 17:35:06+00:00,endpoint,OSSEC v3.7.0,info,CEF:0|OSSEC v3.7.0|SIEM|1.0|100|endpoint|info|...,parkerandre,scheduled_task,can,4878.0,explorer.exe,No additional info,Endpoint scheduled_task can by parkerandre No ...,NaN,NaN,NaN,NaN,0.0.0.0,0.0.0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Eritrea,ead484bdeeed462e83c0d1b831981347d7314c49,Opera/8.26.(Windows 95; is-IS) Presto/2.9.163 ...,3bed87e4-c878-4c08-ad54-7dfae0c558ca,37.64,0.03,SYSTEM,unknown,2020-07-16 17:35:06+00:00,2020-07-16 17:35:06+00:00|unknown|0.0.0.0|0.0.0.0
4,a8bba9b0-994c-4bc9-b614-fbabfc4d3512,2020-07-16 20:16:05+00:00,network,Snort v2.9.20,medium,CEF:0|Snort v2.9.20|SIEM|1.0|100|network|mediu...,NaN,data_exfiltration,NaN,NaN,NaN,MITRE Technique: T1547.001,Network data_exfiltration from 196.184.140.85 ...,NaN,NaN,NaN,NaN,196.184.140.85,193.254.63.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ICMP,257802.0,NaN,NaN,NaN,Botswana,6d34ddd018dcb5b2e5d19df4014b17a2,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,71ec7fb0-cf58-423f-a2be-a36152706530,48.74,0.48,FIREWALL,unknown,2020-07-16 20:16:05+00:00,2020-07-16 20:16:05+00:00|unknown|196.184.140....


,event_id,timestamp,event_type,source,severity,raw_log,user,action,object,process_id,parent_process,additional_info,description,behavioral_analytics,device_type,device_id,firmware_version,src_ip,dst_ip,alert_type,signature_id,category,cloud_service,resource_id,model_id,input_hash,output_hash,src_port,dst_port,protocol,bytes,duration,method,mac_address,meta_geo_location,meta_device_hash,meta_user_agent,meta_session_id,meta_risk_score,meta_confidence,log_source,host,ts_round,merge_key
count,100000,100000,100000,100000,100000,100000,50283,87500,12589,12589.000000,12589,100000,100000,10060,12434,12434,12434.000000,100000,100000,12500,12500,12500,12511,12511,12667,12667,12667,12448.000000,12448.000000,24783,24783.000000,12448.000000,12516,12516,100000,100000,100000,100000,100000.000000,100000.000000,100000,100000,100000,100000
unique,100000,NaN,8,20,6,97222,40855,55,7379,NaN,4,1482,89578,9985,5,12434,NaN,49797,33575,17,6722,5,5,12511,12667,12618,12631,NaN,NaN,10,NaN,NaN,7,12516,245,98886,78721,100000,NaN,NaN,3,1,NaN,99918
top,8f7b1e90-e534-4a0b-b273-c58ce683dafe,NaN,ai,ArcSight v7.4.0,medium,CEF:0|Trellix v10.7.0|SIEM|1.0|100|iot|high| d...,ismith,api_abuse,character,NaN,svchost.exe,No additional info,IoT device Camera physical_tamper No additiona...,"{'baseline_deviation': 0.71, 'entropy': 5.99, ...",Camera,iot-2cfc37c6,NaN,0.0.0.0,0.0.0.0,Zero-Day Exploit,SIG-9496,Evasion,GCP,res-a77d35de,model-7fc6eddc,14d6f2206516fd64d521a4a86c548b90295e5323,4e811b980f5c82b4de55ad88f0416b1649b4c06de18112...,NaN,NaN,TCP,NaN,NaN,key,14:a8:20:86:f6:32,China,61ec2d82f8018598652a53e24a9f809364188a8e939ff8...,Mozilla/5.0 (compatible; MSIE 7.0; Windows NT ...,4ff151bb-32a0-4cfc-9b8e-e38d70faba9b,NaN,NaN,SYSTEM,unknown,NaN,2025-04-05 12:09:18+00:00|unknown|0.0.0.0|0.0.0.0
freq,1,NaN,12667,5134,20639,7,16,2935,15,NaN,3196,57146,217,3,2591,1,NaN,50201,66426,810,8,2556,2545,1,1,5,4,NaN,NaN,3827,NaN,NaN,1833,1,1666,30,69,1,NaN,NaN,62717,100000,NaN,2
mean,NaN,2025-04-16 22:33:25.624699648+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5076.779172,NaN,NaN,NaN,NaN,NaN,NaN,5.993823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7148.251767,7005.676494,NaN,499435.936933,1781.084030,NaN,NaN,NaN,NaN,NaN,NaN,49.999583,0.498681,NaN,NaN,2025-04-16 22:33:25.624699648+00:00,NaN
min,NaN,2020-07-12 21:38:20+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,105.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,2020-07-12 21:38:20+00:00,NaN
25%,NaN,2025-02-23 23:10:19+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2579.000000,NaN,NaN,NaN,NaN,NaN,NaN,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,324.000000,319.000000,NaN,251933.000000,890.000000,NaN,NaN,NaN,NaN,NaN,NaN,36.480000,0.250000,NaN,NaN,2025-02-23 23:10:19+00:00,NaN
50%,NaN,2025-04-12 14:16:27.500000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5037.000000,NaN,NaN,NaN,NaN,NaN,NaN,6.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,645.500000,641.000000,NaN,500400.000000,1768.500000,NaN,NaN,NaN,NaN,NaN,NaN,50.040000,0.500000,NaN,NaN,2025-04-12 14:16:27.500000+00:00,NaN
75%,NaN,2025-05-29 20:27:17.500000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7583.000000,NaN,NaN,NaN,NaN,NaN,NaN,8.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,961.000000,960.000000,NaN,747491.000000,2678.000000,NaN,NaN,NaN,NaN,NaN,NaN,63.600000,0.750000,NaN,NaN,2025-05-29 20:27:17.500000+00:00,NaN
max,NaN,2030-07-10 06:49:21+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9998.000000,NaN,NaN,NaN,NaN,NaN,NaN,10.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65470.000000,65530.000000,NaN,999998.000000,3600.000000,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,1.000000,NaN,NaN,2030-07-10 06:49:21+00:00,NaN


In [40]:
# # Data Cleaning (Required: Step 1.2)

# # Drop rows missing core metadata (timestamp, event_type)
# df = df.dropna(subset=["timestamp", "event_type"])

# # Replace inf with NaN (as in assignment)
# df.replace([np.inf, -np.inf], np.nan, inplace=True)

# # Impute numeric NaNs using median (as in assignment)
# num_cols = df.select_dtypes(include=[np.number]).columns
# df[num_cols] = df[num_cols].fillna(df[num_cols].median(numeric_only=True))

# # Impute categorical NaNs using mode
# cat_cols = [c for c in df.columns if c not in num_cols]
# for c in cat_cols:
#     if df[c].isna().any():
#         mode = df[c].mode(dropna=True)
#         df[c] = df[c].fillna(mode.iloc[0] if len(mode) else "unknown")

# df.shape



# --- Required by assignment: drop missing core metadata, replace inf, impute numeric with median :contentReference[oaicite:2]{index=2}
df = df.dropna(subset=["timestamp", "event_type"])
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# 1) Numeric imputation (median) — OK for numeric features (assignment explicitly suggests this)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
df[num_cols] = df[num_cols].fillna(df[num_cols].median(numeric_only=True))

# 2) Categorical imputation (mode) — BUT SKIP IDS-only / label-like columns
#    Because these are not “general features” and mode-fill will fabricate values.
skip_mode_impute = {
    "category", "alert_type", "signature_id",   # IDS-only fields
    "attack_stage", "label"                     # target/derived labels
}

cat_cols = df.select_dtypes(include=["object", "string"]).columns.tolist()

for c in cat_cols:
    if c in skip_mode_impute:
        continue
    if df[c].isna().any():
        mode = df[c].mode(dropna=True)  # pandas mode docs :contentReference[oaicite:3]{index=3}
        df[c] = df[c].fillna(mode.iloc[0] if len(mode) else "unknown")  # fillna docs :contentReference[oaicite:4]{index=4}


In [41]:
# We need to derive attack_stage first:

# Required context: your dataset doesn't have attack_stage, so we inspect substitutes.
# HF dataset schema says event-specific fields vary; ids_alert has fields like alert_type, category. :contentReference[oaicite:2]{index=2}

print("event_type counts:\n", df["event_type"].value_counts().head(10), "\n")

if "category" in df.columns:
    print("category non-null:", df["category"].notna().sum())
    print("category unique values:\n", df["category"].dropna().unique())
    print("\ncategory value_counts:\n", df["category"].value_counts(dropna=False))
else:
    print("No 'category' column found.")


event_type counts:
 event_type
ai           12667
endpoint     12589
auth         12516
cloud        12511
ids_alert    12500
firewall     12448
iot          12434
network      12335
Name: count, dtype: int64 

category non-null: 12500
category unique values:
 ['Malware' 'Recon' 'Policy' 'Evasion' 'Exploit']

category value_counts:
 category
NaN        87500
Evasion     2556
Policy      2534
Recon       2509
Malware     2453
Exploit     2448
Name: count, dtype: int64


In [ ]:
# # 1) category should only apply to IDS alerts (otherwise it was imputed by mode)
# is_ids = df["event_type"].astype(str).str.lower().eq("ids_alert")
# df.loc[~is_ids, "category"] = pd.NA   # undo the mode-fill for non-IDS rows

# # 2) Map your dataset's IDS category -> assignment's required attack stages
# # Recon & Exploit map directly. The dataset also has Evasion/Malware/Policy.
# # We map them into the closest required stage labels (document this in your report).
# category_to_stage = {
#     "Recon": "recon",
#     "Exploit": "exploit",
#     "Malware": "privilege_escalation",   # proxy: malware commonly enables elevated permissions (MITRE PrivEsc). :contentReference[oaicite:3]{index=3}
#     "Evasion": "lateral_movement",       # proxy: evasion often occurs during movement/pivoting; dataset lacks explicit LM label. :contentReference[oaicite:4]{index=4}
#     "Policy": "benign",                  # policy violations/decisions are not a kill-chain stage in the assignment
# }

# # 3) Build attack_stage
# df["attack_stage"] = "benign"
# df.loc[is_ids, "attack_stage"] = (
#     df.loc[is_ids, "category"]
#       .astype("string")
#       .map(category_to_stage)
#       .fillna("benign")
# )

# df["attack_stage"].value_counts()


# IDS-only mask
is_ids = df["event_type"].astype(str).str.lower().eq("ids_alert")

# Default: everything benign
df["attack_stage"] = "benign"

# Map IDS category into assignment's stages where possible
# Recon → recon (MITRE Reconnaissance TA0043) :contentReference[oaicite:1]{index=1}
# Exploit → exploit (stage name used in the assignment)
cat = df.loc[is_ids, "category"].astype("string")

category_to_stage = {
    "Recon": "recon",
    "Exploit": "exploit",
    "Policy": "benign",   # not a kill-chain stage in the assignment
    "Malware": "benign",  # handled by heuristics (optional) below
    "Evasion": "benign",  # handled by heuristics (optional) below
}

df.loc[is_ids, "attack_stage"] = cat.map(category_to_stage).fillna("benign")

df["attack_stage"].value_counts()


attack_stage
benign     95043
recon       2509
exploit     2448
Name: count, dtype: int64

In [43]:
# Build a text field to search for indicators
text = (
    df["description"].astype("string").fillna("") + " " +
    df["additional_info"].astype("string").fillna("") + " " +
    df["raw_log"].astype("string").fillna("")
)

# Privilege Escalation (MITRE TA0004) :contentReference[oaicite:4]{index=4}
priv_mask = text.str.contains(r"(privilege|escalat|sudo|token|uac|setuid|admin\s+rights)", case=False, regex=True, na=False)
df.loc[priv_mask, "attack_stage"] = "privilege_escalation"

# Lateral Movement (MITRE TA0008) :contentReference[oaicite:5]{index=5}
lm_mask = text.str.contains(r"(lateral|psexec|wmic|smb|remote\s+service|rdp|winrm|pivot)", case=False, regex=True, na=False)
df.loc[lm_mask, "attack_stage"] = "lateral_movement"

# Exfiltration (MITRE TA0010) :contentReference[oaicite:6]{index=6}
exfil_mask = text.str.contains(r"(exfil|data\s*theft|dns\s*tunnel|upload|outbound\s*transfer|leak|T1041|T1048|T1567)", case=False, regex=True, na=False)
df.loc[exfil_mask, "attack_stage"] = "exfiltration"

df["attack_stage"].value_counts()


C:\Users\Carol\AppData\Local\Temp\ipykernel_23944\634537467.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  priv_mask = text.str.contains(r"(privilege|escalat|sudo|token|uac|setuid|admin\s+rights)", case=False, regex=True, na=False)
C:\Users\Carol\AppData\Local\Temp\ipykernel_23944\634537467.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lm_mask = text.str.contains(r"(lateral|psexec|wmic|smb|remote\s+service|rdp|winrm|pivot)", case=False, regex=True, na=False)
C:\Users\Carol\AppData\Local\Temp\ipykernel_23944\634537467.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  exfil_mask = text.str.contains(r"(exfil|data\s*theft|dns\s*tunnel|upload|outbound\s*transfer|leak|T1041|T1048|T1567)", case=False, regex=True, na

attack_stage
benign                  89183
lateral_movement         2502
recon                    2370
exploit                  2294
exfiltration             2276
privilege_escalation     1375
Name: count, dtype: int64

In [44]:
label_map = {
    "benign": 0,
    "recon": 1,
    "exploit": 2,
    "privilege_escalation": 3,
    "lateral_movement": 4,
    "exfiltration": 5
}

df["label"] = df["attack_stage"].map(label_map)

# sanity checks
print("Any unmapped labels? ->", df["label"].isna().any())
print(df["label"].value_counts().sort_index())


Any unmapped labels? -> False
label
0    89183
1     2370
2     2294
3     1375
4     2502
5     2276
Name: count, dtype: int64


In [46]:
# Make sure we don't accidentally include target columns as features later
assert "attack_stage" in df.columns and "label" in df.columns
df[["attack_stage", "label"]].head()


,attack_stage,label
0,benign,0
1,benign,0
2,benign,0
3,benign,0
4,exfiltration,5


In [47]:
# 1) Make sure labels are complete (no NaNs)
print("label NaNs:", df["label"].isna().sum())

# 2) Confirm you only have labels 0..5
print("unique labels:", sorted(df["label"].unique()))


label NaNs: 0
unique labels: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]


In [48]:
# Feature Engineering (Assignment Step 1.4)

session_col = "meta_session_id" 
df = df.sort_values([session_col, "timestamp"]).reset_index(drop=True)

df["time_delta_sec"] = (
    df.groupby(session_col)["timestamp"]
      .diff()
      .dt.total_seconds()
      .fillna(0.0)
      .clip(lower=0.0)
)


In [49]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# IMPORTANT: do NOT include target columns in features
target_cols = {"attack_stage", "label"}

# Categorical features (edit if you want to add/remove)
categorical_candidates = [
    "event_type", "source", "severity", "action", "protocol",
    "log_source", "host"
]
categorical = [c for c in categorical_candidates if c in df.columns and c not in target_cols]

# Numeric features
numeric_candidates = [
    "time_delta_sec",
    "meta_risk_score", "meta_confidence",
    "src_port", "dst_port", "bytes", "duration",
    "process_id", "firmware_version"
]
numeric = [c for c in numeric_candidates if c in df.columns and c not in target_cols]

# Fill missing values for features only (NOT labels)
df[categorical] = df[categorical].fillna("unknown")
df[numeric] = df[numeric].fillna(df[numeric].median(numeric_only=True))

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
        ("num", StandardScaler(), numeric),
    ],
    remainder="drop",
)

X = preprocess.fit_transform(df[categorical + numeric])
y = df["label"].to_numpy()

print("Categorical:", categorical)
print("Numeric:", numeric)
print("X shape:", X.shape, "y shape:", y.shape)


Categorical: ['event_type', 'source', 'severity', 'action', 'protocol', 'log_source', 'host']
Numeric: ['time_delta_sec', 'meta_risk_score', 'meta_confidence', 'src_port', 'dst_port', 'bytes', 'duration', 'process_id', 'firmware_version']
X shape: (100000, 112) y shape: (100000,)


In [50]:
import joblib
from pathlib import Path

joblib.dump(preprocess, MODEL_DIR / "preprocess.joblib")
print("Saved:", MODEL_DIR / "preprocess.joblib")


Saved: d:\INS\Assignment_2\models\preprocess\preprocess.joblib


In [51]:
# --- Required by assignment: fixed length (50–200) :contentReference[oaicite:3]{index=3}
SEQ_LEN = 100
STRIDE  = 100  # set 50 for overlapping windows if you want more sequences

# --- Use session id or sliding windows per host/session :contentReference[oaicite:4]{index=4}
session_col = "meta_session_id"  # you have this column

# Make sure df is sorted for correct sequencing (required) :contentReference[oaicite:5]{index=5}
df = df.sort_values([session_col, "timestamp"]).reset_index(drop=True)

# y is per-event labels (you already have y = df["label"].to_numpy() from Cell 8)
# X is per-event feature matrix (from preprocess.fit_transform in Cell 8)

# Build index windows per session
idx_by_group = df.groupby(session_col).apply(lambda g: g.index.to_numpy()).to_dict()

seq_X = []
seq_mask = []
seq_y = []
seq_groups = []
seq_last_event_id = []

# Helper: get dense rows from sparse or dense X
def get_dense_rows(Xmat, rows):
    block = Xmat[rows]
    if hasattr(block, "toarray"):     # SciPy sparse matrix supports toarray() :contentReference[oaicite:6]{index=6}
        return block.toarray()
    return np.asarray(block)

for gid, idxs in idx_by_group.items():
    idxs = idxs.astype(int)
    for start in range(0, len(idxs), STRIDE):
        w = idxs[start:start + SEQ_LEN]
        if len(w) == 0:
            continue

        # Event vectors (inputs) :contentReference[oaicite:7]{index=7}
        x_block = get_dense_rows(X, w).astype(np.float32)

        # Pad short sequences with PAD token (we pad with zeros) :contentReference[oaicite:8]{index=8}
        pad_len = SEQ_LEN - len(w)
        if pad_len > 0:
            x_pad = np.zeros((pad_len, x_block.shape[1]), dtype=np.float32)
            x_block = np.vstack([x_block, x_pad])

        # Attention mask: 1 for real events, 0 for PAD
        mask = np.zeros((SEQ_LEN,), dtype=np.int32)
        mask[:len(w)] = 1

        # Target label = attack stage of final event (required) :contentReference[oaicite:9]{index=9}
        y_target = int(y[w[-1]])

        seq_X.append(x_block)
        seq_mask.append(mask)
        seq_y.append(y_target)
        seq_groups.append(gid)
        seq_last_event_id.append(df.loc[w[-1], "event_id"])

seq_X = np.stack(seq_X, axis=0)         # (num_seqs, SEQ_LEN, feat_dim)
seq_mask = np.stack(seq_mask, axis=0)   # (num_seqs, SEQ_LEN)
seq_y = np.array(seq_y, dtype=np.int64) # (num_seqs,)

print("Sequences:", seq_X.shape)
print("Masks:", seq_mask.shape)
print("Targets:", seq_y.shape)
print("Unique groups:", len(set(seq_groups)))
print("Label distribution:", dict(zip(*np.unique(seq_y, return_counts=True))))


C:\Users\Carol\AppData\Local\Temp\ipykernel_23944\181559618.py:15: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  idx_by_group = df.groupby(session_col).apply(lambda g: g.index.to_numpy()).to_dict()


Sequences: (100000, 100, 112)
Masks: (100000, 100)
Targets: (100000,)
Unique groups: 100000
Label distribution: {np.int64(0): np.int64(89183), np.int64(1): np.int64(2370), np.int64(2): np.int64(2294), np.int64(3): np.int64(1375), np.int64(4): np.int64(2502), np.int64(5): np.int64(2276)}


In [52]:
# Train/Test split 80/20 without session leakage

from sklearn.model_selection import GroupShuffleSplit

groups = np.array(seq_groups, dtype=object)

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)  # 80/20 :contentReference[oaicite:12]{index=12}
train_idx, test_idx = next(gss.split(seq_X, seq_y, groups=groups))   # group-safe :contentReference[oaicite:13]{index=13}

X_train, X_test = seq_X[train_idx], seq_X[test_idx]
m_train, m_test = seq_mask[train_idx], seq_mask[test_idx]
y_train, y_test = seq_y[train_idx], seq_y[test_idx]

groups_train = set(groups[train_idx])
groups_test  = set(groups[test_idx])

print("Train:", X_train.shape, y_train.shape)
print("Test :", X_test.shape, y_test.shape)
print("Group overlap (must be 0):", len(groups_train.intersection(groups_test)))


Train: (80000, 100, 112) (80000,)
Test : (20000, 100, 112) (20000,)
Group overlap (must be 0): 0


In [54]:
# Save outputs

out_path = PREP_DIR / f"sequences_len{SEQ_LEN}.npz"

np.savez_compressed(
    out_path,
    X_train=X_train, X_test=X_test,
    m_train=m_train, m_test=m_test,
    y_train=y_train, y_test=y_test,
)

# Save group + last_event_id mappings for later reconstruction work
np.save(PREP_DIR / f"seq_groups_len{SEQ_LEN}.npy", np.array(seq_groups, dtype=object), allow_pickle=True)
np.save(PREP_DIR / f"seq_last_event_id_len{SEQ_LEN}.npy", np.array(seq_last_event_id, dtype=object), allow_pickle=True)

print("Saved:", out_path)


Saved: d:\INS\Assignment_2\data\processed\sequences_len100.npz


In [55]:
# Save the unified dataframe so Phase 2 (EDA) can load it
# pandas.DataFrame.to_parquet writes a parquet file. :contentReference[oaicite:3]{index=3}

df.to_parquet(PREP_DIR / "unified_events.parquet", index=False)
df.to_csv(PREP_DIR / "unified_events.csv", index=False)

print("Saved:")
print(" -", PREP_DIR / "unified_events.parquet")
print(" -", PREP_DIR / "unified_events.csv")


Saved:
 - d:\INS\Assignment_2\data\processed\unified_events.parquet
 - d:\INS\Assignment_2\data\processed\unified_events.csv
